In [1]:
#код генерирующий стихи Пушкин
import re
import json
import random
from collections import defaultdict
from tqdm import tqdm
import numpy as np

In [3]:
# -*- coding: utf-8 -*-
"""
Генерация стихов в стиле А.С. Пушкина
n-граммная языковая модель (биграммы и триграммы)
Семинар 2, ИДС / NLP
"""

import re
import random
import json
from collections import defaultdict, Counter
import numpy as np
from tqdm import tqdm

# =============================================================================
# 1. Загрузка и предобработка текста
# =============================================================================

PATH = "pushkin.txt"  # предполагаем, что файл уже скачан и лежит рядом

with open(PATH, encoding="utf-8") as f:
    raw_text = f.read()

# Очень простая очистка
raw_text = raw_text.replace("\xa0", " ").replace("\n", " ").replace("\r", " ")
raw_text = re.sub(r'\s+', ' ', raw_text).strip()

print(f"Длина текста после очистки: {len(raw_text):,} символов")
print("Начало текста:", raw_text[:180], "...\n")

# Разбиваем на предложения (очень грубо)
sentences = [s.strip() for s in re.split(r'[.!?]\s*', raw_text) if len(s.strip()) > 30]
print(f"Получено предложений: {len(sentences)}\n")


# =============================================================================
# 2. Токенизация (только русские слова, нижний регистр)
# =============================================================================

def simple_tokenize(s: str) -> list[str]:
    """Оставляем только кириллические слова"""
    return re.findall(r'[а-яё]+', s.lower())


tokenized = [simple_tokenize(sent) for sent in sentences]
all_tokens = [tok for sent in tokenized for tok in sent]

print(f"Всего токенов: {len(all_tokens):,}")
print(f"Уникальных слов: {len(set(all_tokens)):,}\n")


# =============================================================================
# 3. Подсчёт n-грамм
# =============================================================================

# Униграммы
uni_count = Counter(all_tokens)

# Биграммы
bi_count = defaultdict(int)
bi_prefix_count = defaultdict(int)          # сколько раз встретился первый элемент

for sent in tokenized:
    for i in range(len(sent) - 1):
        pair = (sent[i], sent[i+1])
        bi_count[pair] += 1
        bi_prefix_count[sent[i]] += 1

# Триграммы
tri_count = defaultdict(int)
tri_prefix_count = defaultdict(int)         # сколько раз встретилась биграмма-префикс

for sent in tokenized:
    for i in range(len(sent) - 2):
        triple = (sent[i], sent[i+1], sent[i+2])
        tri_count[triple] += 1
        bi_prefix = (sent[i], sent[i+1])
        tri_prefix_count[bi_prefix] += 1


print(f"Биграмм: {len(bi_count):,}")
print(f"Триграмм: {len(tri_count):,}\n")

print("Топ-8 биграмм:")
for pair, cnt in sorted(bi_count.items(), key=lambda x: -x[1])[:8]:
    print(f"  {pair[0]:<12} → {pair[1]:<12} : {cnt}")

print("\nТоп-6 триграмм:")
for trip, cnt in sorted(tri_count.items(), key=lambda x: -x[1])[:6]:
    print(f"  {trip[0]:<8} {trip[1]:<8} → {trip[2]:<8} : {cnt}")


# =============================================================================
# 4. Функции семплирования
# =============================================================================

def get_candidates_bi(word: str) -> tuple[list[str], list[float]]:
    """Вероятности следующего слова после данного (биграммная модель)"""
    if word not in bi_prefix_count:
        return [], []
    total = bi_prefix_count[word]
    next_words = []
    probs = []
    for (w1, w2), c in bi_count.items():
        if w1 == word:
            next_words.append(w2)
            probs.append(c / total)
    return next_words, probs


def get_candidates_tri(w1: str, w2: str) -> tuple[list[str], list[float]]:
    """Вероятности после двух слов (триграммная модель)"""
    prefix = (w1, w2)
    if prefix not in tri_prefix_count:
        return [], []
    total = tri_prefix_count[prefix]
    next_words = []
    probs = []
    for (a, b, c), cnt in tri_count.items():
        if (a, b) == prefix:
            next_words.append(c)
            probs.append(cnt / total)
    return next_words, probs


def apply_temperature(probs: list[float], temp: float = 1.0) -> list[float]:
    if not probs:
        return []
    if temp <= 0:
        temp = 1e-6
    logits = np.log(np.array(probs) + 1e-12)
    logits = logits / temp
    exp_logits = np.exp(logits)
    return (exp_logits / exp_logits.sum()).tolist()


def sample_word(cands: list[str], probs: list[float], temp: float = 1.0) -> str:
    if not cands:
        # запасной вариант — самое частое слово
        return uni_count.most_common(1)[0][0]
    
    probs = apply_temperature(probs, temp)
    return random.choices(cands, weights=probs, k=1)[0]


# =============================================================================
# 5. Генерация четверостишия
# =============================================================================

def generate_poem(
    model: str = "tri",           # "bi" или "tri"
    start_word: str = None,
    temp: float = 0.9,
    words_per_line: int = 8,
    lines: int = 4
) -> list[str]:
    
    if start_word is None:
        start_word = random.choice(list(uni_count.keys()))
    
    poem_tokens = [start_word]
    
    while len(poem_tokens) < words_per_line * lines:
        if model == "tri" and len(poem_tokens) >= 2:
            cands, pr = get_candidates_tri(poem_tokens[-2], poem_tokens[-1])
        else:
            cands, pr = get_candidates_bi(poem_tokens[-1])
        
        next_w = sample_word(cands, pr, temp)
        poem_tokens.append(next_w)
    
    # Разбиваем на строки
    poem_lines = []
    for i in range(0, len(poem_tokens), words_per_line):
        line_tokens = poem_tokens[i : i + words_per_line]
        poem_lines.append(" ".join(line_tokens))
    
    poem_lines = poem_lines[:lines]
    
    # Очень наивная попытка рифмы (повторяем последнее слово 2-й строки в 4-й)
    if len(poem_lines) >= 4 and random.random() < 0.65:
        try:
            rhyme_word = poem_lines[1].split()[-1]
            last_line_words = poem_lines[3].split()[:-1]
            poem_lines[3] = " ".join(last_line_words + [rhyme_word])
        except:
            pass  # если строка слишком короткая — пропускаем
    
    return poem_lines


# =============================================================================
# 6. Запуск и демонстрация
# =============================================================================

random.seed(43)   # чуть другой сид

print("\n" + "="*70)
print("ГЕНЕРАЦИЯ СТИХОВ\n")

print("Вариант 1: биграммы, начало «луна», температура 1.1")
poem_bi = generate_poem(model="bi", start_word="луна", temp=1.1)
for ln in poem_bi:
    print("  " + ln)

print("\nВариант 2: триграммы, начало «мой», температура 0.75")
poem_tri = generate_poem(model="tri", start_word="мой", temp=0.75)
for ln in poem_tri:
    print("  " + ln)

print("\nВариант 3: триграммы, случайный старт, высокая температура")
poem_random = generate_poem(model="tri", temp=1.6)
for ln in poem_random:
    print("  " + ln)


# Сохранение моделей (для отчёта или повторного использования)
with open("bi_model.json", "w", encoding="utf-8") as f:
    json.dump({f"{a}_{b}": cnt for (a,b), cnt in bi_count.items()}, f, ensure_ascii=False, indent=1)

with open("tri_model.json", "w", encoding="utf-8") as f:
    json.dump({f"{a}_{b}_{c}": cnt for (a,b,c), cnt in tri_count.items()}, f, ensure_ascii=False, indent=1)

print("\nМодели сохранены в bi_model.json и tri_model.json")

Длина текста после очистки: 659,217 символов
Начало текста: Жил-был поп, Толоконный лоб. Пошел поп по базару Посмотреть кой-какого товару. Навстречу ему Балда. Идет, сам не зная куда. «Что, батька, так рано поднялся? Чего ты взыскался?» Поп ...

Получено предложений: 5784

Всего токенов: 98,887
Уникальных слов: 24,479

Биграмм: 76,268
Триграмм: 83,769

Топ-8 биграмм:
  и            → в            : 183
  и            → с            : 111
  я            → не           : 72
  и            → не           : 54
  и            → на           : 52
  я            → в            : 49
  и            → ты           : 47
  мой          → друг         : 44

Топ-6 триграмм:
  в        последний → раз      : 16
  ты       помнишь  → ли       : 12
  с        сватьей  → бабой    : 11
  сватьей  бабой    → бабарихой : 11
  а        ткачиха  → с        : 10
  ткачиха  с        → поварихой : 10

ГЕНЕРАЦИЯ СТИХОВ

Вариант 1: биграммы, начало «луна», температура 1.1
  луна в истине блаженство я со груди сво